# Lister Report  

In [ ]:
#importing libraries needed to desplay results

In [ ]:
from requests import HTTPError

import cobra
import pandas as pd
import plotly.express as px
import seaborn as sns

from Bio.KEGG import REST
from matplotlib import rcParams
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Set default figure size
rcParams["figure.figsize"] = 10, 10

## Wildtype/Control Model

Import the `iCardio` model for "wildtype."

In [ ]:
#Loading in Icardio as the control to allow us to comapare fluxes after changing constraints 

In [ ]:
wildtype= cobra.io.read_sbml_model("2021-06-11_HeartModel.sbml.xml")

In [ ]:
#Solving the wildtpe model using optimize function.
#This is done using Gurobi Optimizer, a mathematical programming solver. 
#It maximixes or minimises an objective function by finding the best values across certain constraints.

Solve the "wildtype" model.

In [ ]:
wildtype.optimize()

In [ ]:
#Inspect glucose transport flux in the solved model.

In [ ]:
glc_trans_wt = wildtype.reactions.get_by_id("RCR40464")     # glucose transport
glc_na_trans_wt = wildtype.reactions.get_by_id("RCR41035")  # glucose transport with sodium

In [ ]:
glc_trans_wt

In [ ]:
glc_na_trans_wt

Checking fluxes through both glucose transport steps

In [ ]:
glc_trans_wt.flux

In [ ]:
glc_na_trans_wt.flux

## Glucose Transport Perturbation

Shutting off all glucose import in the perturbed model. 
First importing the `iCardio` model.

In [ ]:
perturbed = cobra.io.read_sbml_model("2021-06-11_HeartModel.sbml.xml")

In [ ]:
#Setting the lower and upper bounds for both glucose transport steps to zero.

In [ ]:
glc_trans_pt = perturbed.reactions.get_by_id("RCR40464")     # glucose transport
glc_trans_pt.lower_bound, glc_trans_pt.upper_bound = 0, 0

glc_na_trans_pt = perturbed.reactions.get_by_id("RCR41035")  # glucose transport with sodium
glc_na_trans_pt.lower_bound, glc_na_trans_pt.upper_bound = 0, 0

In [ ]:
#solving the perturbed model

In [ ]:
perturbed.optimize()

In [ ]:
# Inspecting the glucose transport steps after changing constraints .

In [ ]:
glc_trans_pt

In [ ]:
glc_na_trans_pt

## Plotting flux differences

Using a scatterplot to show changes in flux through each step in the model, given the perturbation above. 
First we generate a dataframe containing the flux for each reaction, in the "wildtype" and "perturbed" solutions.

In [ ]:
rxn_names = [_.name for _ in wildtype.reactions]
rxn_wt_flux = [_.flux for _ in wildtype.reactions]
rxn_pt_flux = [_.flux for _ in perturbed.reactions]

fluxes = pd.DataFrame({"wildtype": rxn_wt_flux, "perturbed": rxn_pt_flux}, index=rxn_names)
fluxes

In [ ]:
#Using the  package `seaborn` to generate a static image 

In [ ]:
sns.scatterplot(x="wildtype", y="perturbed", data=fluxes);

In [ ]:
#create a new column to hold reaction "state" after perturbation

In [ ]:
fluxes["state"] = ""

# Assign states to each row in the dataframe
def assign_state(row):
    """Assign a state to a row (reaction)"""
    if row["wildtype"] == 0 and row["perturbed"] != 0:
        return "activated"
    elif row["wildtype"] != 0 and row["perturbed"] == 0:
        return "deactivated"
    elif row["wildtype"] == 0 and row["perturbed"] == 0:
        return "zero flux"
    elif abs(row["perturbed"]/row["wildtype"]) > 1.1:
        return "increased flux"
    elif abs(row["perturbed"]/row["wildtype"]) < 1/1.1:
        return "decreased flux"    
    return "unchanged"

fluxes["state"] = fluxes.apply(assign_state, axis=1)

In [ ]:
# creating scatterplot with new column "state"

In [ ]:
sns.scatterplot(data=fluxes, x="wildtype", y="perturbed", hue="state");

## Interactive Plot

`plot.ly` allows us to produce an interactive plot where we can define what happens on mouseover events and get more detail about individual reactions

**Move your mouse pointer over some of the data points**

In [ ]:
fig = px.scatter(fluxes, x="wildtype", y="perturbed", color="state", hover_data=[fluxes.index])
fig.show()

Using two data resources : the Chalmers `Human-GEM` model and an interface to the KEGG database, which we'll get to using Biopython. This will allow a useful reaction name to replace the 'RCR' numbers.

## Labelling points
Obtaining the Chalmers `Human-GEM` model from [the GitHub repo](https://github.com/SysBioChalmers/Human-GEM) and getting the `model/reactions.tsv` file from this repository.
KEGG is a computer representation of the biological system, it consists of molecular building blocks of genes and proteins and chemical substances. It is used to understand the functions and utilities of the biological system.

In [ ]:
reactions = pd.read_csv("reactions.tsv", sep="\t")
reactions

The `RCR` numbers are in the column `rxnRatconID`, and the KEGG reaction IDs are in `rxnKEGGID`. 
Now wanting a list of reaction names from the KEGG database.

For example, `R00754` maps to information at its [reaction page](https://www.kegg.jp/entry/R00754). 
We could capture any piece of information from this entry using the Biopython REST interface.
This allows for easy access to a range of KEGG datab

In [ ]:
print(REST.kegg_get("R00754").read())

But we need some kind of function to parse this information (other functions are possible).

In [ ]:
def parse_rxn_definition(rxn_data):
    """Return the Definition of a KEGG reaction"""
    for line in [_.strip() for _ in rxn_data]:
        if line.startswith("DEFINITION"):
            label, info = line.split(" ", 1)
            return info.strip()
        
parse_rxn_definition(REST.kegg_get("R00754"))

In [ ]:
#Updating the `fluxes` dataframe with the KEGG ID, and the reaction definition.

In [ ]:
fluxes["KEGGID"] = ""  # creating a new column to hold KEGG IDs

# Assigning KEGG IDs to each row in the dataframe
def assign_kegg(row):
    """Assign KEGG ID to a row (reaction)"""
    match = reactions.loc[reactions["rxnRatconID"] == row.name]
    if not len(match):
        return "no_kegg_id"
    keggid = match["rxnKEGGID"].item()
    if keggid != "nan":
        return keggid
    return "no_kegg_id"

fluxes["KEGGID"] = fluxes.apply(assign_kegg, axis=1)
fluxes

Now using KEGG to get the reaction definition for each reaction that has a valid KEGG ID
some of the KEGG IDs appear to be out of date and will fail.

In [ ]:
#Didnt run this, instead, ran the pickle file at the start of 'Report_2' as this code takes time 

In [ ]:
fluxes["definition"] = ""  # create a new column to hold KEGG reaction definition info

# Assign KEGG reaction definitions
def assign_definition(row):
    """Assign KEGG definition info to a row (reaction)"""
    if not str(row["KEGGID"]).startswith("R"):
        return "no_kegg_id"
    else:
        tries = 0
        while tries < 10:
            try:
                tries += 1
                kegg_data = REST.kegg_get(row["KEGGID"])
                return parse_rxn_definition(kegg_data)
            except Exception as e:
                print(f"{e} for {row['KEGGID']}")
    
    return "no_kegg_id"
    

fluxes["definition"] = fluxes.apply(assign_definition, axis=1)
fluxes